<a href="https://colab.research.google.com/github/jorgemarcoes/R-Clustering/blob/main/R_Clustering_on_UCR_Archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LIBRARIES

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
#comment if you do not need to install the sktime package
!pip install sktime
from sktime.datasets import load_UCR_UEA_dataset

In [ ]:
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#UCR ARCHIVE

In [ ]:
#@title Datasets
benchmark_datasets = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken',
'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration',
'CinCECGTorso', 'Coffee', 'Computers', 'CricketX', 'CricketY',
'CricketZ', 'Crop', 'DiatomSizeReduction',
'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
'DodgerLoopWeekend', 'Earthquakes', 'ECG5000', 'ECGFiveDays',
'ElectricDevices', 'EOGHorizontalSignal', 'EOGVerticalSignal',
'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords',
'Fish', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi',
'GunPoint', 'GunPointAgeSpan', 'GunPointMaleVersusFemale',
'GunPointOldVersusYoung', 'Ham', 'HandOutlines', 'Haptics',
'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain',
'InsectEPGSmallTrain', 'InsectWingbeatSound', 'ItalyPowerDemand',
'LargeKitchenAppliances', 'Lightning7', 'Mallat', 'Meat',
'MedicalImages', 'MelbournePedestrian',
'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect',
'MiddlePhalanxTW', 'MixedShapesRegularTrain',
'MixedShapesSmallTrain', 'MoteStrain',
'NonInvasiveFetalECGThorax1', 'NonInvasiveFetalECGThorax2',
'OliveOil', 'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme',
'PigAirwayPressure', 'PigArtPressure', 'PigCVP', 'Plane',
'PowerCons', 'ProximalPhalanxOutlineAgeGroup',
'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW',
'RefrigerationDevices', 'Rock', 'ScreenType', 'SemgHandGenderCh2',
'SemgHandMovementCh2', 'SemgHandSubjectCh2', 'ShapeletSim',
'ShapesAll', 'SmallKitchenAppliances', 'SmoothSubspace',
'SonyAIBORobotSurface1', 'SonyAIBORobotSurface2',
'StarLightCurves', 'Strawberry', 'SwedishLeaf', 'Symbols',
'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2',
'Trace', 'TwoLeadECG', 'TwoPatterns', 'UMD',
'UWaveGestureLibraryAll', 'UWaveGestureLibraryX',
'UWaveGestureLibraryY', 'UWaveGestureLibraryZ', 'Wine',
'WordSynonyms', 'Worms', 'WormsTwoClass', 'Yoga']

rocket_development_datasets = ['Beef', 'BirdChicken',
'Car', 'CricketX','CricketY','CricketZ','DistalPhalanxTW','ECG5000',
'FiftyWords','Fish','Haptics','Herring','InsectWingbeatSound','ItalyPowerDemand',
'LargeKitchenAppliances','Lightning7','Meat','MedicalImages',
'MiddlePhalanxOutlineAgeGroup','OSULeaf','OliveOil','Phoneme',
'Plane','ProximalPhalanxOutlineCorrect','ProximalPhalanxTW','ScreenType','ShapeletSim',
'Strawberry','SwedishLeaf','SyntheticControl','ToeSegmentation1','Trace','UWaveGestureLibraryY',
'WordSynonyms','Worms','Yoga']

validation_datasets = np.setdiff1d(benchmark_datasets, rocket_development_datasets)



In [ ]:
def sktime_to_numpy(dataset_name):
  """
   This function retrieves a dataset from the UCR archive using the sktime library,
   including both the train and test sets, and converts it to a numpy format.
   It also addresses an issue with the Melbourne Pedestrian dataset
  """
  X, y = load_UCR_UEA_dataset(name=dataset_name)
  if(dataset_name == 'MelbournePedestrian'): #Missing data in this dataser
    X.drop(3030,inplace=True)
    X.drop(3304,inplace=True)
    y = np.delete(y,3030)
    y = np.delete(y,3304)
  X_numpy = []
  for index, row in X.iterrows():
    X_numpy.append(row.dim_0.to_numpy())
  X_numpy_ = np.array(X_numpy)
  return X_numpy_.astype(np.float32),y

#FEATURE EXTRACTION

In [ ]:
#@title Modified feature extraction stage of Minirocket

# Modified version of the Minirocket algorithm:
# Angus Dempster, Daniel F Schmidt, Geoffrey I Webb

# MiniRocket: A Very Fast (Almost) Deterministic Transform for Time Series
# Classification

# https://arxiv.org/abs/2012.08791

########MODIFICATIONS############
#1. We randomly permute the indices array used for choosing the kernel components (see array indices and compare with original)
#2. We randomly permute the array of quantiles in the function called fit

#These changes successfully eliminate artificial periodic patterns in the transformed input.


from numba import njit, prange, vectorize

@njit("float32[:](float32[:,:],int32[:],int32[:],float32[:])", fastmath = True, parallel = False, cache = True)
def _fit_biases(X, dilations, num_features_per_dilation, quantiles):

    num_examples, input_length = X.shape

    # equivalent to:
    # >>> from itertools import combinations
    # >>> indices = np.array([_ for _ in combinations(np.arange(9), 3)], dtype = np.int32)
    ###MODIFICATION
    indices = np.array((
       1, 3, 6, 1, 2, 7, 1, 2, 3, 0, 2, 3, 1, 4, 5, 0, 1, 3, 3, 5, 6, 0,
       1, 2, 2, 5, 8, 1, 3, 7, 0, 1, 8, 4, 6, 7, 0, 1, 4, 3, 4, 6, 0, 4,
       5, 2, 6, 7, 5, 6, 7, 0, 1, 6, 4, 5, 7, 4, 7, 8, 1, 6, 8, 0, 2, 6,
       5, 6, 8, 2, 5, 7, 0, 1, 7, 0, 7, 8, 0, 3, 5, 0, 3, 7, 2, 3, 8, 2,
       3, 4, 1, 4, 6, 3, 4, 5, 0, 3, 8, 4, 5, 8, 0, 4, 6, 1, 4, 8, 6, 7,
       8, 4, 6, 8, 0, 3, 4, 1, 3, 4, 1, 5, 7, 1, 4, 7, 1, 2, 8, 0, 6, 7,
       1, 6, 7, 1, 3, 5, 0, 1, 5, 0, 4, 8, 4, 5, 6, 0, 2, 5, 3, 5, 7, 0,
       2, 4, 2, 6, 8, 2, 3, 7, 2, 5, 6, 2, 4, 8, 0, 2, 7, 3, 6, 8, 2, 3,
       6, 3, 7, 8, 0, 5, 8, 1, 2, 6, 2, 3, 5, 1, 5, 8, 3, 6, 7, 3, 4, 7,
       0, 4, 7, 3, 5, 8, 2, 4, 5, 1, 2, 5, 2, 7, 8, 2, 4, 6, 0, 5, 6, 3,
       4, 8, 0, 6, 8, 2, 4, 7, 0, 2, 8, 0, 3, 6, 5, 7, 8, 1, 5, 6, 1, 2,
       4, 0, 5, 7, 1, 3, 8, 1, 7, 8
    ), dtype = np.int32).reshape(84, 3)

    num_kernels = len(indices)
    num_dilations = len(dilations)

    num_features = num_kernels * np.sum(num_features_per_dilation)

    biases = np.zeros(num_features, dtype = np.float32)

    feature_index_start = 0

    for dilation_index in range(num_dilations):

        dilation = dilations[dilation_index]
        padding = ((9 - 1) * dilation) // 2

        num_features_this_dilation = num_features_per_dilation[dilation_index]

        for kernel_index in range(num_kernels):

            feature_index_end = feature_index_start + num_features_this_dilation

            _X = X[np.random.randint(num_examples)]

            A = -_X          # A = alpha * X = -X
            G = _X + _X + _X # G = gamma * X = 3X

            C_alpha = np.zeros(input_length, dtype = np.float32)
            C_alpha[:] = A

            C_gamma = np.zeros((9, input_length), dtype = np.float32)
            C_gamma[9 // 2] = G

            start = dilation
            end = input_length - padding

            for gamma_index in range(9 // 2):

                C_alpha[-end:] = C_alpha[-end:] + A[:end]
                C_gamma[gamma_index, -end:] = G[:end]

                end += dilation

            for gamma_index in range(9 // 2 + 1, 9):

                C_alpha[:-start] = C_alpha[:-start] + A[start:]
                C_gamma[gamma_index, :-start] = G[start:]

                start += dilation

            index_0, index_1, index_2 = indices[kernel_index]

            C = C_alpha + C_gamma[index_0] + C_gamma[index_1] + C_gamma[index_2]

            biases[feature_index_start:feature_index_end] = np.quantile(C, quantiles[feature_index_start:feature_index_end])

            feature_index_start = feature_index_end

    return biases

def _fit_dilations(input_length, num_features, max_dilations_per_kernel):

    num_kernels = 84

    num_features_per_kernel = num_features // num_kernels
    true_max_dilations_per_kernel = min(num_features_per_kernel, max_dilations_per_kernel)
    multiplier = num_features_per_kernel / true_max_dilations_per_kernel

    max_exponent = np.log2((input_length - 1) / (9 - 1))
    dilations, num_features_per_dilation = \
    np.unique(np.logspace(0, max_exponent, true_max_dilations_per_kernel, base = 2).astype(np.int32), return_counts = True)
    num_features_per_dilation = (num_features_per_dilation * multiplier).astype(np.int32) # this is a vector

    remainder = num_features_per_kernel - np.sum(num_features_per_dilation)
    i = 0
    while remainder > 0:
        num_features_per_dilation[i] += 1
        remainder -= 1
        i = (i + 1) % len(num_features_per_dilation)

    return dilations, num_features_per_dilation

# low-discrepancy sequence to assign quantiles to kernel/dilation combinations
def _quantiles(n):
    return np.array([(_ * ((np.sqrt(5) + 1) / 2)) % 1 for _ in range(1, n + 1)], dtype = np.float32)

def fit(X, num_features = 10_000, max_dilations_per_kernel = 32):

    _, input_length = X.shape

    num_kernels = 84

    dilations, num_features_per_dilation = _fit_dilations(input_length, num_features, max_dilations_per_kernel)

    num_features_per_kernel = np.sum(num_features_per_dilation)

    quantiles = _quantiles(num_kernels * num_features_per_kernel)

    ###MODIFICATION
    quantiles = np.random.permutation(quantiles)

    biases = _fit_biases(X, dilations, num_features_per_dilation, quantiles)


    return dilations, num_features_per_dilation, biases

# _PPV(C, b).mean() returns PPV for vector C (convolution output) and scalar b (bias)
@vectorize("float32(float32,float32)", nopython = True, cache = True)
def _PPV(a, b):
    if a > b:
        return 1
    else:
        return 0

@njit("float32[:,:](float32[:,:],Tuple((int32[:],int32[:],float32[:])))", fastmath = True, parallel = True, cache = True)
def transform(X, parameters):

    num_examples, input_length = X.shape

    #LOS BIASES LOS HEMOS TRANSFORMADO A TRAVÉS DE LOS QUANTILES
    dilations, num_features_per_dilation, biases = parameters

    # equivalent to:
    # >>> from itertools import combinations
    # >>> indices = np.array([_ for _ in combinations(np.arange(9), 3)], dtype = np.int32)
    ###MODIFICATION
    indices = np.array((
       1, 3, 6, 1, 2, 7, 1, 2, 3, 0, 2, 3, 1, 4, 5, 0, 1, 3, 3, 5, 6, 0,
       1, 2, 2, 5, 8, 1, 3, 7, 0, 1, 8, 4, 6, 7, 0, 1, 4, 3, 4, 6, 0, 4,
       5, 2, 6, 7, 5, 6, 7, 0, 1, 6, 4, 5, 7, 4, 7, 8, 1, 6, 8, 0, 2, 6,
       5, 6, 8, 2, 5, 7, 0, 1, 7, 0, 7, 8, 0, 3, 5, 0, 3, 7, 2, 3, 8, 2,
       3, 4, 1, 4, 6, 3, 4, 5, 0, 3, 8, 4, 5, 8, 0, 4, 6, 1, 4, 8, 6, 7,
       8, 4, 6, 8, 0, 3, 4, 1, 3, 4, 1, 5, 7, 1, 4, 7, 1, 2, 8, 0, 6, 7,
       1, 6, 7, 1, 3, 5, 0, 1, 5, 0, 4, 8, 4, 5, 6, 0, 2, 5, 3, 5, 7, 0,
       2, 4, 2, 6, 8, 2, 3, 7, 2, 5, 6, 2, 4, 8, 0, 2, 7, 3, 6, 8, 2, 3,
       6, 3, 7, 8, 0, 5, 8, 1, 2, 6, 2, 3, 5, 1, 5, 8, 3, 6, 7, 3, 4, 7,
       0, 4, 7, 3, 5, 8, 2, 4, 5, 1, 2, 5, 2, 7, 8, 2, 4, 6, 0, 5, 6, 3,
       4, 8, 0, 6, 8, 2, 4, 7, 0, 2, 8, 0, 3, 6, 5, 7, 8, 1, 5, 6, 1, 2,
       4, 0, 5, 7, 1, 3, 8, 1, 7, 8
    ), dtype = np.int32).reshape(84, 3)


    num_kernels = len(indices)
    num_dilations = len(dilations)

    num_features = num_kernels * np.sum(num_features_per_dilation)

    features = np.zeros((num_examples, num_features), dtype = np.float32)

    for example_index in prange(num_examples):

        _X = X[example_index]

        A = -_X          # A = alpha * X = -X
        G = _X + _X + _X # G = gamma * X = 3X

        feature_index_start = 0

        for dilation_index in range(num_dilations):

            _padding0 = dilation_index % 2

            dilation = dilations[dilation_index]
            padding = ((9 - 1) * dilation) // 2

            num_features_this_dilation = num_features_per_dilation[dilation_index]

            C_alpha = np.zeros(input_length, dtype = np.float32)
            C_alpha[:] = A

            C_gamma = np.zeros((9, input_length), dtype = np.float32)
            C_gamma[9 // 2] = G

            start = dilation
            end = input_length - padding

            for gamma_index in range(9 // 2):

                C_alpha[-end:] = C_alpha[-end:] + A[:end]
                C_gamma[gamma_index, -end:] = G[:end]

                end += dilation

            for gamma_index in range(9 // 2 + 1, 9):

                C_alpha[:-start] = C_alpha[:-start] + A[start:]
                C_gamma[gamma_index, :-start] = G[start:]

                start += dilation

            for kernel_index in range(num_kernels):

                feature_index_end = feature_index_start + num_features_this_dilation

                _padding1 = (_padding0 + kernel_index) % 2

                index_0, index_1, index_2 = indices[kernel_index]

                C = C_alpha + C_gamma[index_0] + C_gamma[index_1] + C_gamma[index_2]

                if _padding1 == 0:
                    for feature_count in range(num_features_this_dilation):
                        features[example_index, feature_index_start + feature_count] = _PPV(C, biases[feature_index_start + feature_count]).mean()
                else:
                    for feature_count in range(num_features_this_dilation):
                        features[example_index, feature_index_start + feature_count] = _PPV(C[padding:-padding], biases[feature_index_start + feature_count]).mean()

                feature_index_start = feature_index_end

    return features

#EXPERIMENT

In [ ]:
#@title R-Clustering evaluated across UCR dataset

# Define the number of features or kernels (500 by default based on previous development experiments)
num_features = 500

# Define the destination to save results, default is the current folder, but it can be modified
destination = "Results.xlsx"

# Initialize a list to store the results
results_list = []
i = 0

# Iterate over the validation datasets
for dataset in validation_datasets:
    print(dataset)

    # Convert the sktime datasets to numpy arrays
    X, Y = sktime_to_numpy(dataset)


    #######################################################################################
    # STAGE 1: DATA TRANSFORMATION WITH THE MODIFIED MINIROCKET ALGORITHM
    #######################################################################################
    # Fit and transform data according to the modified minirocket model with the given number of features
    parameters = fit(X=X, num_features=num_features)
    transformed_data = transform(X=X, parameters=parameters)

    #######################################################################################
    # STAGE 2: DIMENSIONALITY REDUCTION WITH PCA
    #######################################################################################
    # Standardize the data to have mean=0 and variance=1 for better performance of machine learning models
    sc = StandardScaler()
    X_std = sc.fit_transform(transformed_data)

    # Implement Principal Component Analysis (PCA) to reduce dimensions and focus on relevant features
    pca = PCA().fit(X_std)

    # Find the optimal number of dimensions based on the criterion that the explained variance ratio is less than 0.01
    optimal_dimensions = np.argmax(pca.explained_variance_ratio_ < 0.01)

    # Apply PCA with the optimal number of dimensions
    pca_optimal = PCA(n_components=optimal_dimensions)
    transformed_data_pca = pca_optimal.fit_transform(X_std)
    print("Dimensions: ", transformed_data_pca.shape[1])

    #######################################################################################
    # STAGE 3: CLUSTERING ALGORITHM: K-Means
    #######################################################################################
    # Get the number of clusters of the dataset
    num_clusters = len(np.unique(Y))
    # Apply KMeans clustering on the transformed data
    labels_pred = KMeans(n_clusters=num_clusters, n_init=10).fit_predict(transformed_data_pca)


    #######################################################################################
    # EVALUATION
    #######################################################################################

    # Evaluate the clustering using Adjusted Rand Index (ARI) score
    score = metrics.adjusted_rand_score(labels_true=Y, labels_pred=labels_pred)
    print("ARI: ", score)
    print("\n")

    # Store each result in a dictionary to create a pandas dataframe later
    results_row = dict(Dataset=dataset, ari_score=score)
    results_list.append(results_row)

    # Save interim results every 5 iterations
    i += 1
    if i % 5 == 0:
        results_df = pd.DataFrame(results_list)
        results_df.to_excel(destination)

# Convert the results list to a pandas DataFrame and save to an Excel file
results_df = pd.DataFrame(results_list)
results_df.to_excel(destination)
